# Bioimage Model Zoo Core  Example notebook

This notebook shows how to interact with the `bioimageio.core` programmatically to explore, load, use, and export content from the [BioImage Model Zoo](https://bioimage.io).


quick links:
- [Create an input sample for a given model](#create_sample_for_model)

## Activate human readable output error messages and load dependencies

### Running local

If you are running the notebook locally you need to pip install the following dependencies into your local environment. Make sure to restart your notebook kernel after installing dependencies.

```console
pip install bioimageio.core==0.8.0 torch==2.5.1 matplotlib==3.10.1
```

### Running in colab

If you are running this notebook through colab, there is no need to install dependencies manually, just run the following cell.

In [ ]:
import os

# install dependencies if running in colab
if os.getenv("COLAB_RELEASE_TAG"):
    %pip install bioimageio.core==0.8.0 torch==2.5.1 matplotlib==3.10.1

### Install necessary dependencies

In [ ]:
import torch
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from imageio.v2 import imread

from bioimageio.spec.utils import download

import bioimageio.core

from bioimageio.spec.pretty_validation_errors import (
    enable_pretty_validation_errors_in_ipynb,
)

# Improve readiblity of validation errors
enable_pretty_validation_errors_in_ipynb()

# Function to display input and prediction output images
def show_images(sample_tensor, prediction_tensor):
    input_array = sample_tensor.members["input0"].data

    # Check for the number of channels to enable display
    input_array = np.squeeze(input_array)
    if len(input_array.shape) > 2:
        input_array = input_array[0]

    output_array = prediction_tensor.members["output0"].data

    # Check for the number of channels to enable display
    output_array = np.squeeze(output_array)
    if len(output_array.shape) > 2:
        output_array = output_array[0]

    plt.figure()
    ax1 = plt.subplot(1, 2, 1)
    ax1.set_title("Input")
    ax1.axis("off")
    plt.imshow(input_array)
    ax2 = plt.subplot(1, 2, 2)
    ax2.set_title("Prediction")
    ax2.axis("off")
    plt.imshow(output_array)
    plt.show()

## Load a model

### Inspect available models in the Bioimage Model Zoo

Go to https://bioimage.io to browser available models

### Load model from the BioImage Model Zoo

`bioimage.io` resources may be identified via their bioimage.io __ID__, e.g. "affable-shark" or the [__DOI__](https://doi.org/) of their [__Zenodo__](https://zenodo.org/) backup.

Both of these options may be version specific ("affable-shark/1" or a version specific [__Zenodo__](https://zenodo.org/) backup [__DOI__](https://doi.org/)).

Alternatively, any rdf.yaml source, single file or in a .zip, may be loaded by providing its __local path__ or __URL__.

**Note** -- For more detailed information about loading a model, inspecting a model's metadata and creating your own model visit [bioimageio.spec package example notebook](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb).

In [ ]:
model_source = "affable-shark"
#model_source = "10.5281/zenodo.6287342"
#model_source = "https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/draft/files/rdf.yaml"

`load_model_description` is a function of the `bioimageio.spec` package, but as it is a sub-package of `bioimageio.core` it can also be called from it by `bioimageio.core.load_model_description`.

To learn more about the functionalities of the `bioimageio.spec` package, see the [bioimageio.spec package example notebook](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb), also available as a [Google Colab](https://colab.research.google.com/github/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb) notebook.

In [ ]:
from bioimageio.core import load_model_description
from bioimageio.core import test_model

model = load_model_description(model_source)
test_summary = test_model(model) #Test model to check environment compatibility  
assert test_summary.status == "passed", test_summary.display() 

## Running a prediction

`bioimageio.core` implements the functionality to run a prediction with models described in the `bioimage.io` format.

This includes functions to run predictions on `numpy.ndarray`/`xarray.DataArray` as input and convenience functions to run predictions for images stored on disc.

### `predict` function 

The most direct way to run a model prediction is using the `bioimageio.core` convenience function `predict`, to which you pass the model you want to predict with and the input you want to predict on, along with optional arguments with default values. The input can be a range of types including; `numpy.ndarray`, `xarray.DataArray`, bioimageio `Tensor` or `pathlib.Path` to data. The union of all these types are contained within the `TensorSource` object.

`bioimageio.core.Tensors/xarray.DataArrays` are like numpy arrays, but they have annotated axes.

However, sometimes a model requires multiple inputs to be passed in at once (e.g a raw data input and a mask input). To easily facilitate this inputs can be passed in as a mapping between input id and `TensorSource`. For example defining mulitple `numpy.ndarray` inputs could be done manually done as follows:

```console
inputs = {"raw_input": np.array([1,2,3]), "mask_input": np.array()}
```

To make things more convinent multiple inputs can instead be passed via a bioimageio `Sample` object

In [ ]:
from bioimageio.core import predict  # , predict_many
from bioimageio.core import Sample

# predict_many(model=model, inputs=[sample])

prediction: Sample = predict(model=model, inputs=sample)

# show the prediction result
show_images(sample, prediction)

### Load the test image and convert into a tensor

In [ ]:
from bioimageio.spec.model import v0_5
from bioimageio.spec.utils import load_array

assert isinstance(model, v0_5.ModelDescr)
input_image = load_array(model.inputs[0].test_tensor)
print(f"array shape: {input_image.shape}")

Create a `Tensor` (light wrapper around an `xarray.DataArray`) from the test input image. 

`bioimageio.core.Tensors/xarray.DataArrays` are like numpy arrays, but they have annotated axes.

The axes are used to validate that the axes of the input image match the axes expected by the model.

In [ ]:
from bioimageio.core import Tensor

test_input_tensor = Tensor.from_numpy(input_image, dims=model.inputs[0].axes)

# print the axis annotations ('dims') and the shape of the input array
print(f"tensor shape: {test_input_tensor.tagged_shape}")

A collection of tensors is called a `Sample`.

In the case of the `affable-shark` model it only has one input, but for models with multiple inputs a `Sample` includes a tensor for each input.

In [ ]:
from bioimageio.core import Sample

sample = Sample(members={"raw": test_input_tensor}, stat={}, id="sample-from-numpy")

sample

`bioimageio.core` provides the helper function `create_sample_for_model` to automatically create the `Sample` for the given model.
<a id='create_sample_for_model'></a>

In [ ]:
from bioimageio.core.digest_spec import create_sample_for_model
from bioimageio.spec.utils import download

input_paths = {ipt.id: download(ipt.test_tensor).path for ipt in model.inputs}
print(f"input paths: {input_paths}")
assert isinstance(model, v0_5.ModelDescr)
sample = create_sample_for_model(
    model=model, inputs=input_paths, sample_id="my_demo_sample"
)

sample

There is also  a helper function `get_test_inputs` to directly import the test input sample for a given model.

In [ ]:
from bioimageio.core.digest_spec import get_test_inputs

test_sample = get_test_inputs(model)

test_sample

### 3.2. Create a prediciton pipeline

The `prediction_pipeline` function is used to run a prediction with a given model.

It applies the __pre-processing__, if indicated in the model rdf.yaml, runs __inference__ with the model and applies the __post-processing__, again if specified in the model rdf.yaml.

The `devices` argument can be used to specify which device(s), CPU, a single GPU, or multiple GPUs (not implemented yet), to use for inference with the model.

The default is `devices=None`, this will use a __GPU__ if available, otherwise it uses the __CPU__.


The `weight_format` argument can be used to specify which of the model's available weight formats to use.

The deafult is `weight_format=None`, this will use the weight format with highest priority (as defined by bioimageio.core).



In [ ]:
from bioimageio.core import create_prediction_pipeline

devices = None
weight_format = None

prediction_pipeline = create_prediction_pipeline(
    model, devices=devices, weight_format=weight_format
)

Use the new prediction pipeline to run a prediction for the previously loaded test image.

The prediction pipeline returns a `Sample` object, which will be displayed.

In [ ]:
prediction: Sample = prediction_pipeline.predict_sample_without_blocking(sample)

# show the prediction result
show_images(sample, prediction)

### 3.3. Prediction without a PredicitionPipeline

`bioimageio.core` has two convenience functions `predict` and `predict_many` which allow the prediction of images without creating a `PredictionPipeline`.

In [ ]:
from bioimageio.core import predict  # , predict_many

# predict_many(model=model, inputs=[sample])

prediction: Sample = predict(model=model, inputs=sample)

# show the prediction result
show_images(sample, prediction)

### 3.3. Recover input and output tensors as numpy arrays

This example code shows how to recover the image information from the input and output tensors as numpy arrays.

In [ ]:
np_input_list = []
np_output_list = []

# iterate over the number of tensors inside the input sample
for ipt in range(len(sample.members.keys())):
    input_array = sample.members[f"input{ipt}"].data

    # Check for the number of channels to enable display
    input_array = np.squeeze(input_array)
    if len(input_array.shape) > 2:
        input_array = input_array[0]

    np_input_list.append(input_array)


# iterate over the number of tensors inside the output prediction
for out in range(len(prediction.members.keys())):
    output_array = prediction.members[f"output{ipt}"].data

    # Check for the number of channels to enable display
    output_array = np.squeeze(output_array)
    if len(output_array.shape) > 2:
        output_array = output_array[0]

    np_output_list.append(output_array)

plt.imshow(np_input_list[0])